## Preprocessing

In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Copy of Original Pandas Dataframe created from CSV
application_df_copy = application_df.copy()

In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [27]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [28]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Set a cutoff value
cutoff_value = 1000 # Increased to 1000 from original model's 400

# Get the value counts for APPLICATION_TYPE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Identify application types to be replaced based on the cutoff value
application_types_to_replace = list(application_type_counts[application_type_counts < cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())


T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64


In [29]:
# Look at CLASSIFICATION value counts for binning

# Display value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Set a cutoff value
classification_cutoff_value = 1000 # Increased to 1000 from original model's 400

# Identify classifications to be replaced based on the cutoff value
classifications_to_replace = list(classification_counts[classification_counts < classification_cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [30]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]
print(classification_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [31]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Display CLASSIFICATION value counts
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Set a cutoff value
classification_cutoff_value = 1000 # Increased to 1000 from original model's 400

# Identify classifications to be replaced based on the cutoff value
classifications_to_replace = list(classification_counts[classification_counts < classification_cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [32]:
# Convert categorical data to numeric with `pd.get_dummies` - This is One Hot Encoding
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], drop_first=True)
application_df_encoded.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,1,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0


In [33]:
# Split our preprocessed data into our features and target arrays

# Define features X and target variable y
X = application_df_encoded.drop("IS_SUCCESSFUL", axis=1)
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Test for Success
print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)

X_train Shape: (25724, 33)
X_test Shape: (8575, 33)
y_train Shape: (25724,)
y_test Shape: (8575,)


In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# How many dimensions does X_train_scaled and X_test_scaled have
print(f"X_train_scaled dim: {X_train_scaled.shape}")
print(f"X_test_scaled dim: {X_test_scaled.shape}")


X_train_scaled dim: (25724, 33)
X_test_scaled dim: (8575, 33)


## Compile, Train and Evaluate the Model #2


In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# This second model is also utilizing Keras, a deep neural network with three (vs two) hidden layers, each having 5 neurons with ReLU activation. The input layer has 33 (vs 37) features, and the output layer has 1 neuron with a sigmoid activation function, indicating that it's designed for binary classification.

# Creating a Keras Sequential Model
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=33))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 5)                 170       
                                                                 
 dense_9 (Dense)             (None, 5)                 30        
                                                                 
 dense_10 (Dense)            (None, 5)                 30        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 236 (944.00 Byte)
Trainable params: 236 (944.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# Compile the second model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [38]:
# Train the second model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6517 - accuracy: 0.6389
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5881 - accuracy: 0.7116
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5751 - accuracy: 0.7226
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5701 - accuracy: 0.7230
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5678 - accuracy: 0.7245
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5666 - accuracy: 0.7256
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5656 - accuracy: 0.7253
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5647 - accuracy: 0.7261
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5641 - accuracy: 0.7280
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accura

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5579 - accuracy: 0.7304 - 549ms/epoch - 2ms/step
Loss: 0.5579259395599365, Accuracy: 0.7303789854049683


## Compile, Train and Evaluate the Model #3



In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# This third model is also utilizing Keras, a deep neural network with four (vs three) hidden layers, each having 10 (vs 5) neurons with ReLU activation. The input layer has 33 features, and the output layer has 1 neuron with a sigmoid activation function, indicating that it's designed for binary classification.

# Creating a Keras Sequential Model
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=33))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                340       
                                                                 
 dense_13 (Dense)            (None, 10)                110       
                                                                 
 dense_14 (Dense)            (None, 10)                110       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 571 (2.23 KB)
Trainable params: 571 (2.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Compile the third model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [42]:
# Train the third model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=50) # vs 100 epochs


Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.6114 - accuracy: 0.6772
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5686 - accuracy: 0.7197
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5624 - accuracy: 0.7252
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7258
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5580 - accuracy: 0.7269
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5570 - accuracy: 0.7282
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7277
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7297
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7289
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7292

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5626 - accuracy: 0.7237 - 534ms/epoch - 2ms/step
Loss: 0.5625733733177185, Accuracy: 0.7237317562103271
